In [1]:
%load_ext autoreload
%autoreload 2

from pysat.formula import CNF
from compiler.entrypoint import Max3satQaoaCompiler

formula = CNF(from_file="./instances/uf20-01.cnf")

print(formula)

CNF(from_string='c This Formular is generated by mcnf\nc\nc    horn? no\nc    forced? no\nc    mixed sat? no\nc    clause length = 3\nc\np cnf 20 91\n4 -18 19 0\n3 18 -5 0\n-5 -8 -15 0\n-20 7 -16 0\n10 -13 -7 0\n-12 -9 17 0\n17 19 5 0\n-16 9 15 0\n11 -5 -14 0\n18 -10 13 0\n-3 11 12 0\n-6 -17 -8 0\n-18 14 1 0\n-19 -15 10 0\n12 18 -19 0\n-8 4 7 0\n-8 -9 4 0\n7 17 -15 0\n12 -7 -14 0\n-10 -11 8 0\n2 -15 -11 0\n9 6 1 0\n-11 20 -17 0\n9 -15 13 0\n12 -7 -17 0\n-18 -2 20 0\n20 12 4 0\n19 11 14 0\n-16 18 -4 0\n-1 -17 -19 0\n-13 15 10 0\n-12 -14 -13 0\n12 -14 -7 0\n-7 16 10 0\n6 10 7 0\n20 14 -16 0\n-19 17 11 0\n-7 1 -20 0\n-5 12 15 0\n-4 -9 -13 0\n12 -11 -7 0\n-5 19 -8 0\n1 16 17 0\n20 -14 -15 0\n13 -4 10 0\n14 7 10 0\n-5 9 20 0\n10 1 -19 0\n-16 -15 -1 0\n16 3 -11 0\n-15 -10 4 0\n4 -15 -3 0\n-10 -16 11 0\n-8 12 -5 0\n14 -6 12 0\n1 6 11 0\n-13 -5 -1 0\n-7 -2 12 0\n1 -20 19 0\n-2 -13 -8 0\n15 18 4 0\n-11 14 9 0\n-6 -15 -2 0\n5 -12 -15 0\n-6 17 5 0\n-13 5 -19 0\n20 -1 14 0\n9 -17 15 0\n-5 19 -18 0

In [10]:
compiler = Max3satQaoaCompiler(formula)
program = compiler.compile_single_layer()
fidelity, duration = program.avg_fidelity(), program.duration()
print("Number of Instructions: ", len(program.instructions), ", Fidelity: ", fidelity, ", Duration ", duration)
print(program.count_ops())

Number of Instructions:  1416 , Fidelity:  0.047747363205786646 , Duration  10659.821214530695
{'u3': 652, 'cz': 196, 'ccz': 140}


In [8]:
from utils.hamiltonians import Max3satHamiltonian
from utils.qaoa import QAOA
from qiskit import transpile

basis_gates = ["u3", "id", "cz"]
hamiltonian = Max3satHamiltonian(formula=formula)
qaoa = QAOA(hamiltonian)
circuit, cost_params, mixer_params = qaoa.naive_qaoa_circuit(1)
bound_circuit = circuit.assign_parameters({cost_params: [0.2512], mixer_params: [0.1235]})
transpiled_circuit = transpile(bound_circuit, basis_gates=basis_gates, optimization_level=3)
print(bound_circuit.count_ops())
ops = transpiled_circuit.count_ops()
print("Fidelity: ", (0.999 ** ops["u3"]) * (0.995 ** ops["cz"]))
print(ops)


OrderedDict([('cx', 590), ('rz', 231), ('h', 20), ('rx', 20)])
Fidelity:  0.06394125013650653
OrderedDict([('u3', 544), ('cz', 440)])
